In [1]:
import pyspark
sc = SparkContext.getOrCreate()


dfDetails = spark.read.csv(
    "details.csv", header=True, mode="DROPMALFORMED",sep  = ","
)
dfDetails.show(1)

dfInvoices = spark.read.csv(
    "invoices.csv", header=True, mode="DROPMALFORMED",sep  = ","
)
dfInvoices.show(1)

dfProduct = spark.read.json(
    "products.json"
)
dfProduct.show(1)

dfStore = spark.read.json(
    "stores.json"
)
dfStore.show(1)





print('done  import datasets')

+--------+-------+--------+-----+
| Invoice|Product|Quantity|Price|
+--------+-------+--------+-----+
|01069586|   Peas|     1.1| 0.33|
+--------+-------+--------+-----+
only showing top 1 row

+--------+----------+--------+--------+
| Invoice|      Date|Customer|   Store|
+--------+----------+--------+--------+
|01069586|1996-11-28|   03200|Carlisle|
+--------+----------+--------+--------+
only showing top 1 row

+----+-------+-----+----+
|cost|   name|price|unit|
+----+-------+-----+----+
|0.08|Tomatos| 0.15|  kg|
+----+-------+-----+----+
only showing top 1 row

+--------+
|    name|
+--------+
|Aberdeen|
+--------+
only showing top 1 row

done  import datasets


In [2]:
import pyspark.sql.functions as fn



dfProfit = dfDetails.join(dfProduct, dfDetails.Product == dfProduct.name, how='left').drop(dfDetails.Price)

dfProfit.withColumn('diff', (fn.col('price')-fn.col('cost'))*fn.col('Quantity')).show()





+--------+-----------+--------+----+-----------+-----+------+--------------------+
| Invoice|    Product|Quantity|cost|       name|price|  unit|                diff|
+--------+-----------+--------+----+-----------+-----+------+--------------------+
|01069586|       Peas|     1.1|0.16|       Peas|  0.3|    kg|               0.154|
|01069586|       Tuna|       5|0.63|       Tuna|  1.0|   can|                1.85|
|07088623|     Grapes|    0.94|0.28|     Grapes| 0.46|    kg|              0.1692|
|07088623|    Raisins|       1|0.23|    Raisins| 0.45|    kg|                0.22|
|07088623|   Sausages|     0.5| 1.8|   Sausages|  2.4|    kg| 0.29999999999999993|
|07088623|      Pears|    1.83|0.24|      Pears| 0.33|    kg| 0.16470000000000004|
|07088623|    Lettuce|    4.22|0.86|    Lettuce| 1.25|    kg|              1.6458|
|07088623|Cooking Oil|     1.0|0.73|Cooking Oil|  1.2|liters|                0.47|
|08083256|       Tuna|       2|0.63|       Tuna|  1.0|   can|                0.74|
|080

In [3]:
dfTotals = dfInvoices.join(dfDetails, dfInvoices.Invoice == dfDetails.Invoice, how="left").drop(dfDetails.Invoice)

dfTotals = dfTotals.groupby('Invoice').agg(fn.sum('Price').alias('total'))

dfTotals.show(2)

+--------+------------------+
| Invoice|             total|
+--------+------------------+
|01017570|28.699999999999996|
|01046160|12.649999999999999|
+--------+------------------+
only showing top 2 rows



In [4]:
dfTotals.sort(fn.col('total'), ascending=False).show()

+--------+------------------+
| Invoice|             total|
+--------+------------------+
|08001410|            273.06|
|03022748|             242.6|
|02065806|199.00000000000003|
|04038373|194.57999999999998|
|01002195|            182.16|
|00016021|             181.3|
|00046997|             173.7|
|07007638|173.42000000000002|
|10047557|170.47000000000003|
|06013374|            170.26|
|03093846|            168.48|
|02014749|            167.09|
|05066742|            166.73|
|03094271|             165.7|
|01023743|165.04999999999998|
|06059247|            164.97|
|04055805|            164.06|
|00025887|            163.85|
|05042782|163.59000000000003|
|07077425|            161.35|
+--------+------------------+
only showing top 20 rows



In [5]:
dfTotalSold = dfProduct.join(dfDetails, dfProduct.name == dfDetails.Product, how="left").drop(dfProduct.price)

dfTotalSold.show(1)

quantitySold =dfTotalSold.groupby(['name']).agg(fn.sum('quantity').alias('total_sold'))
quantitySold.show(1)

netSold = dfTotalSold.groupby(['name']).agg(fn.sum('price').alias('net_profit'))
netSold.show(1)

totalSold = dfTotalSold.groupby(['name']).agg(fn.sum('price').alias('net'),fn.sum('cost').alias('cost')).withColumn('diff', fn.col('net')-fn.col('cost').alias('total proffit'))
totalSold.show(1)

+----+-------+----+--------+-------+--------+-----+
|cost|   name|unit| Invoice|Product|Quantity|Price|
+----+-------+----+--------+-------+--------+-----+
|0.08|Tomatos|  kg|10049060|Tomatos|     2.7| 0.41|
+----+-------+----+--------+-------+--------+-----+
only showing top 1 row

+-------+------------------+
|   name|        total_sold|
+-------+------------------+
|Peaches|1616.4699999999998|
+-------+------------------+
only showing top 1 row

+-------+------------------+
|   name|        net_profit|
+-------+------------------+
|Peaches|1519.5300000000007|
+-------+------------------+
only showing top 1 row

+-------+------------------+-----------------+-----------------+
|   name|               net|             cost|             diff|
+-------+------------------+-----------------+-----------------+
|Peaches|1519.5300000000007|667.0000000000064|852.5299999999943|
+-------+------------------+-----------------+-----------------+
only showing top 1 row



In [6]:
quantitySold.sort(fn.col('total_sold'), ascending=False ).show(1)

totalSold.sort(fn.col('diff'), ascending=False).show(1)

netSold.sort(fn.col('net_profit'),ascending=False).show(1)

+-------+----------+
|   name|total_sold|
+-------+----------+
|Diapers|   33160.0|
+-------+----------+
only showing top 1 row

+-------+-----------------+------------------+-----------------+
|   name|              net|              cost|             diff|
+-------+-----------------+------------------+-----------------+
|Diapers|79583.99999999975|1692.6000000000317|77891.39999999972|
+-------+-----------------+------------------+-----------------+
only showing top 1 row

+-------+-----------------+
|   name|       net_profit|
+-------+-----------------+
|Diapers|79583.99999999975|
+-------+-----------------+
only showing top 1 row



In [7]:
dfStoresJoined = dfStore.join(dfInvoices, dfStore.name == dfInvoices.Store, how = "left")

dfStoresJoined = dfStoresJoined.join(dfDetails, dfStoresJoined.Invoice == dfDetails.Invoice)

dfStoresJoined.show(1)

dfStoresJoined.groupBy(['Store']).agg(fn.sum('Price').alias('total_sold')).sort(fn.col('total_sold'), asending=False).show(1)

dfStoresJoined = dfStoresJoined.join(dfProduct, dfStoresJoined.Product == dfProduct.name, how = "left").drop(dfProduct.price)

dfStoresJoined.groupBy(['Store']).agg(fn.sum('price').alias('total_sold'), fn.sum('cost').alias('cost')).withColumn('diff', fn.col('Total_sold')-fn.col('cost').alias('total proffit')).sort(fn.col('diff'),asending=False).show(1)

+--------+--------+----------+--------+--------+--------+-------+--------+-----+
|    name| Invoice|      Date|Customer|   Store| Invoice|Product|Quantity|Price|
+--------+--------+----------+--------+--------+--------+-------+--------+-----+
|Aberdeen|08033512|1996-11-29|   01554|Aberdeen|08033512|  Pears|    2.26| 0.75|
+--------+--------+----------+--------+--------+--------+-------+--------+-----+
only showing top 1 row

+-------+------------------+
|  Store|        total_sold|
+-------+------------------+
|Swansea|11644.710000000006|
+-------+------------------+
only showing top 1 row

+-------+------------------+------------------+-----------------+
|  Store|        total_sold|              cost|             diff|
+-------+------------------+------------------+-----------------+
|Swansea|11644.710000000006|1823.0499999999874|9821.660000000018|
+-------+------------------+------------------+-----------------+
only showing top 1 row



In [8]:
dfStoreList = dfStore.join(dfInvoices, dfStore.name == dfInvoices.Store)

dfStoreList = dfStoreList.join(dfDetails, dfStoreList.Invoice == dfDetails.Invoice)


dfStoreList = dfStoreList.groupBy(['Store','Product']).agg(fn.count('Product').alias('cant'))
dfStoreList.show(1)

+-------+-------+----+
|  Store|Product|cant|
+-------+-------+----+
|Grimsby|   Peas|  69|
+-------+-------+----+
only showing top 1 row



In [9]:
dfInvoices.groupby(['Customer']).agg(fn.when(fn.count('Customer') > 1, fn.lit(False) ).otherwise(True).alias('has_returned')).show(1)

+--------+------------+
|Customer|has_returned|
+--------+------------+
|   06660|        true|
+--------+------------+
only showing top 1 row



In [10]:
dfCustomerList = dfInvoices.join(dfDetails, dfInvoices.Invoice == dfDetails.Invoice).drop(dfInvoices.Invoice)

dfCustomerList = dfCustomerList.groupBy(['Customer','Invoice']).agg(fn.sum('Price').alias('sum')).groupBy(['Customer']).agg(fn.sum('sum').alias('sum')).sort(fn.col('sum'),ascending=False)

dfCustomerList.show(1)

+--------+------+
|Customer|   sum|
+--------+------+
|   00066|296.01|
+--------+------+
only showing top 1 row



In [11]:
dfCustomerList = dfInvoices.join(dfDetails, dfInvoices.Invoice == dfDetails.Invoice).drop(dfInvoices.Invoice)


dfCustomerList = dfCustomerList.groupBy(['Customer','Invoice']).agg(fn.sum('Quantity').alias('sum')).groupBy(['Customer']).agg(fn.sum('sum').alias('sum')).sort(fn.col('sum'),ascending=False)


dfCustomerList.show(1)

+--------+------------------+
|Customer|               sum|
+--------+------------------+
|   05531|171.98999999999998|
+--------+------------------+
only showing top 1 row

